In [1]:
import pandas as pd
import numpy as np
### read the data 
fields = ['star_rating', 'review_body']

test = pd.read_csv("amazon_reviews_us_Kitchen_v1_00.tsv", sep = '\t',usecols=fields,error_bad_lines = False)

In [2]:
### keep the balanced dataset 
### sample 50000 per star_rating from the data
sample_size = 50000 ### here we use 10000 for testing 
df = test.groupby('star_rating').apply(lambda x: x.sample(sample_size))

In [3]:
df["label"] = df.star_rating.apply(lambda x: 1 if x > 3.0 else( 2 if x < 3.0 else 3))

In [4]:
df =df.rename(columns ={"review_body": "review"})

In [5]:
import re

df["review"] = df["review"].astype(str)

### Data cleaning and preprocssing 
df["review"] = df["review"].str.lower()

## remove HTML 
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
df["review"] = df["review"].apply(lambda x :remove_tags(x))

## remove URL
df["review"] = df["review"].str.replace(r's*https?://S+(s+|$)', ' ').str.strip()

def remove_extraS(text):
    return re.sub(' +', ' ', text)

df["review"] = df["review"].apply(lambda x: remove_extraS(x))

def contractionfunction(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

df["review"] = df["review"].apply(lambda x: contractionfunction(x))

## remove non-alphabetical characters
df["review"] = df["review"].str.replace('[^a-zA-Z]', ' ')

<ipython-input-5-6d9c790aa9e0>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["review"] = df["review"].str.replace(r's*https?://S+(s+|$)', ' ').str.strip()
<ipython-input-5-6d9c790aa9e0>:41: FutureWarning: The default value of regex will change from True to False in a future version.
  df["review"] = df["review"].str.replace('[^a-zA-Z]', ' ')


In [6]:
### preprocessing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

df['review'] = df['review'].apply(word_tokenize)
#df.head()

stop_words = set(stopwords.words('english'))
df['review'] = df['review'].apply(lambda x: [word for word in x if word not in stop_words])

In [7]:
import gensim.downloader as api

pretrained_model = api.load("word2vec-google-news-300")

In [8]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load("word2vec.model")

In [94]:
def review2tensor(x,model1=True):
    ### we only need 50 reviews
    max_review = 50
    ### create a matrix 
    review = torch.zeros(max_review,1,300)
    
    index = 0
    for word in x:
       ### using pretrained model 
        if model1:
            try:
                vec = pretrained_model[word]
                index += 1
            except:
                x.remove(word)  # exclude the non existing word from our vec 
                continue
        else: # using my own model
            try:
                vec = w2v_model.wv[word]
                index += 1
                
            except:
                x.remove(word)
                continue
        
        temp = torch.zeros(1, 300)
        vec = torch.from_numpy(np.array(vec))
        temp[0] = vec
        ## if we have 50 reviews already break the loop 
        if index == max_review:
            break
        
        else:
            review[index-1][0] = temp
            
    final_tensor = torch.nan_to_num(review)
        
    return final_tensor 
        


In [95]:
b = df.review.iloc[0]

In [96]:
the = review2tensor(b,model1=True)

In [97]:
the.size()

torch.Size([50, 1, 300])

# Binary case : pretrained model (googel news 300) 

In [74]:
binary_df = df.loc[df["label"] <3]

In [101]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 50
n_label = 2

rnn = RNN(300, n_hidden, n_label)

In [102]:
### prepare training
def labelfromoutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return  category_i


In [103]:
b = binary_df["review"].iloc[3]

In [104]:
input_ = review2tensor(b,model1=True)
hidden= torch.zeros(1, n_hidden)
output, next_hidden = rnn(input_[0], hidden)

In [105]:
print(labelfromoutput(output))
print(output)

1
tensor([[-0.7009, -0.6854]], grad_fn=<LogSoftmaxBackward>)


In [106]:
criterion = nn.NLLLoss()

learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()
    ### to prevent grandient explode
    torch.nn.utils.clip_grad_norm_(rnn.parameters(),0.5)

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [107]:
binary_df["label"] = binary_df["label"].apply(lambda x: 0 if x ==1 else 1)

<ipython-input-107-6ad3ccdd6d7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_df["label"] = binary_df["label"].apply(lambda x: 0 if x ==1 else 1)


In [108]:
all_data = list(zip(binary_df.review.values, binary_df.label))

In [109]:
### train test split 
from sklearn.model_selection import train_test_split
import random
random.shuffle(all_data)
train_dataset, test_dataset = train_test_split(all_data, test_size=0.2,random_state=42)

In [110]:
#test_dataset

In [111]:
import time
import math


#### train using train dataset 
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

correct = 0
index = 0
for whole_item in train_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    review_tensor = review2tensor(review, model1= True)
    
    
    #category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(label_tensor, review_tensor)
    current_loss += loss
    index +=1 
    ### 
    y_pred = labelfromoutput(output)
    if y_pred  == label:
        correct +=1 
        
    if index % 5000 == 0:
        #print('%d %d%% (%s) %.4f ' % (index, index / len(train_dataset) * 100, timeSince(start), loss/5000))
        #current_loss = 0
        
        print("{},complete {:4f}%, {}s,the current averge loss is {:4f}".format(index,index/len(train_dataset)*100, timeSince(start),current_loss/index))
        

5000,complete 3.125000%, 0m 37ss,the current averge loss is 0.700042
10000,complete 6.250000%, 1m 13ss,the current averge loss is 0.709490
15000,complete 9.375000%, 1m 50ss,the current averge loss is 0.722315
20000,complete 12.500000%, 2m 26ss,the current averge loss is 0.732321
25000,complete 15.625000%, 3m 3ss,the current averge loss is 0.740009
30000,complete 18.750000%, 3m 39ss,the current averge loss is 0.747519
35000,complete 21.875000%, 4m 14ss,the current averge loss is 0.763829
40000,complete 25.000000%, 4m 49ss,the current averge loss is 0.820406
45000,complete 28.125000%, 5m 24ss,the current averge loss is 0.948682
50000,complete 31.250000%, 6m 0ss,the current averge loss is 1.050588
55000,complete 34.375000%, 6m 35ss,the current averge loss is 1.130361
60000,complete 37.500000%, 7m 10ss,the current averge loss is 1.192259
65000,complete 40.625000%, 7m 45ss,the current averge loss is 1.238549
70000,complete 43.750000%, 8m 20ss,the current averge loss is 1.270532
75000,comple

In [112]:
## evalueating the result and get test accuracy 
# Keep track of correct guesses in a confusion matrix
n_label = 2
confusion = torch.zeros(n_label, n_label)
#n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
      

    return output

for whole_item in test_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    review_tensor = review2tensor(review, model1= True)
    output = evaluate(review_tensor)
    
    label_pred = labelfromoutput(output)
    confusion[label][label_pred] += 1


In [113]:
### to calculate test accuracy using confusion table

confusion

tensor([[16908.,  3070.],
        [ 5147., 14875.]])

In [114]:
### get the test accuracy from confusion table 

def accuracy_cf(table):
    diagonal_sum = table.trace()
    
    sum_of_all_elements = table.sum()
    
    return diagonal_sum/sum_of_all_elements


In [116]:
print("RNN : Binary Case with pretrained model (google news 300)| The Test Accuracy is {} ".format(accuracy_cf(confusion)))

RNN : Binary Case with pretrained model (google news 300)| The Test Accuracy is 0.7945749759674072 


# Tenary Case: pretrained_model (google_news_300)

In [117]:
df

star_rating  \
star_rating                        
1.0         3333330          1.0   
            2710317          1.0   
            4674244          1.0   
            513833           1.0   
            3548916          1.0   
...                          ...   
5.0         2283852          5.0   
            87780            5.0   
            4245030          5.0   
            744933           5.0   
            2756911          5.0   

                                                                review  label  
star_rating                                                                    
1.0         3333330  [unit, never, worked, gives, eo, message, appa...      2  
            2710317  [gave, one, star, nothing, sticks, however, tu...      2  
            4674244  [loved, microwave, loved, look, buttons, acros...      2  
            513833               [way, smaller, thought, would, happy]      2  
            3548916  [kettle, waste, money, cord, attached, kettle,...      2  
...                                                                ...    ...  
5.0         2283852  [exactly, looking, handles, x, easily, simple,...      1  
            87780               [strong, works, well, glad, purchased]      1  
            4245030  [finally, decent, set, pans, yahoo, pans, clea...      1  
            744933   [love, rice, cooker, replaced, basic, rice, co...      1  
            2756911  [makes, juicing, citrus, breeze, easy, use, ea...      1  

[250000 rows x 3 columns]

In [118]:
df["y"] = df["label"].apply(lambda x: 0 if x ==1 else (1 if x==2 else 2) )

In [119]:
ternary_data = list(zip(df.review.values, df.y.values))

In [120]:
#ternary_data[:1]

In [121]:
### train test split 
from sklearn.model_selection import train_test_split
import random
random.shuffle(ternary_data)
train_dataset, test_dataset = train_test_split(ternary_data, test_size=0.2,random_state=42)

In [122]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 50
n_label = 3

rnn = RNN(300, n_hidden, n_label)

In [123]:
criterion = nn.NLLLoss()

learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()
    ### to prevent grandient explode
    torch.nn.utils.clip_grad_norm_(rnn.parameters(),0.5)

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [124]:
import time
import math


#### train using train dataset 
current_loss = 0
#all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

correct = 0
index = 0
for whole_item in train_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    review_tensor = review2tensor(review, model1= True)
    
    
    #category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(label_tensor, review_tensor)
    current_loss += loss
    index +=1 
    ### 
    y_pred = labelfromoutput(output)
    if y_pred  == label:
        correct +=1 
        
    if index % 5000 == 0:
        #print('%d %d%% (%s) %.4f ' % (index, index / len(train_dataset) * 100, timeSince(start), loss/5000))
        #current_loss = 0
        
        print("{},complete:{}%, {}s,the current averge loss is {:4f}".format(index,index/len(train_dataset)*100, timeSince(start),current_loss/index))
        

5000,complete:2.5%, 0m 37ss,the current averge loss is 1.063436
10000,complete:5.0%, 1m 13ss,the current averge loss is 1.066940
15000,complete:7.5%, 1m 49ss,the current averge loss is 1.067553
20000,complete:10.0%, 2m 25ss,the current averge loss is 1.065846
25000,complete:12.5%, 3m 2ss,the current averge loss is 1.068998
30000,complete:15.0%, 3m 38ss,the current averge loss is 1.068099
35000,complete:17.5%, 4m 14ss,the current averge loss is 1.068760
40000,complete:20.0%, 4m 50ss,the current averge loss is 1.068945
45000,complete:22.5%, 5m 26ss,the current averge loss is 1.067864
50000,complete:25.0%, 6m 4ss,the current averge loss is 1.066530
55000,complete:27.500000000000004%, 6m 40ss,the current averge loss is 1.065656
60000,complete:30.0%, 7m 16ss,the current averge loss is 1.065141
65000,complete:32.5%, 7m 52ss,the current averge loss is 1.064845
70000,complete:35.0%, 8m 27ss,the current averge loss is 1.064866
75000,complete:37.5%, 9m 3ss,the current averge loss is 1.065845
800

In [125]:
## evalueating the result and get test accuracy 
# Keep track of correct guesses in a confusion matrix
n_label = 3
confusion = torch.zeros(n_label, n_label)
#n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
      

    return output

for whole_item in test_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    review_tensor = review2tensor(review, model1= True)
    output = evaluate(review_tensor)
    
    label_pred = labelfromoutput(output)
    confusion[label][label_pred] += 1


In [126]:
### get the test accuracy from confusion table 

def accuracy_cf(table):
    diagonal_sum = table.trace()
    
    sum_of_all_elements = table.sum()
    
    return diagonal_sum/sum_of_all_elements


In [127]:
confusion

tensor([[14817.,  4790.,   358.],
        [ 3382., 16416.,   368.],
        [ 3982.,  5628.,   259.]])

In [128]:
print("RNN : Ternary case with pretrained model : the Test Accuracy is  {:4f}".format(accuracy_cf(confusion)))


RNN : Ternary case with pretrained model : the Test Accuracy is  0.629840


# Binary Case : My Own word2vec model 

In [129]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 50
n_label = 2

rnn = RNN(300, n_hidden, n_label)

In [130]:
criterion = nn.NLLLoss()

learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()
    ### to prevent grandient explode
    torch.nn.utils.clip_grad_norm_(rnn.parameters(),0.5)

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [131]:
all_data = list(zip(binary_df.review.values, binary_df.label))

In [132]:
### train test split 
from sklearn.model_selection import train_test_split
import random
random.shuffle(all_data)
train_dataset, test_dataset = train_test_split(all_data, test_size=0.2,random_state=42)

In [133]:
import time
import math


#### train using train dataset 
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

correct = 0
index = 0
for whole_item in train_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    ## using my own word2vec model set model1 = False
    review_tensor = review2tensor(review, model1= False)
    
    output, loss = train(label_tensor, review_tensor)
    current_loss += loss
    index +=1 
    ### 
    y_pred = labelfromoutput(output)
    if y_pred  == label:
        correct +=1 
        
    if index % 5000 == 0:
        #print('%d %d%% (%s) %.4f ' % (index, index / len(train_dataset) * 100, timeSince(start), loss/5000))
        #current_loss = 0
        
        print("{},complete {:4f}%, {}s,the current averge loss is {:4f}".format(index,index/len(train_dataset)*100, timeSince(start),current_loss/index))
        

5000,complete 3.125000%, 0m 36ss,the current averge loss is 0.701494
10000,complete 6.250000%, 1m 12ss,the current averge loss is 0.727929
15000,complete 9.375000%, 1m 48ss,the current averge loss is 0.728947
20000,complete 12.500000%, 2m 24ss,the current averge loss is 0.728648
25000,complete 15.625000%, 3m 0ss,the current averge loss is 0.724527
30000,complete 18.750000%, 3m 37ss,the current averge loss is 0.739134
35000,complete 21.875000%, 4m 12ss,the current averge loss is 0.785923
40000,complete 25.000000%, 4m 48ss,the current averge loss is 0.915387
45000,complete 28.125000%, 5m 23ss,the current averge loss is 1.025527
50000,complete 31.250000%, 6m 0ss,the current averge loss is 1.124239
55000,complete 34.375000%, 6m 38ss,the current averge loss is 1.197335
60000,complete 37.500000%, 7m 15ss,the current averge loss is 1.254508
65000,complete 40.625000%, 7m 53ss,the current averge loss is 1.291524
70000,complete 43.750000%, 8m 30ss,the current averge loss is 1.313684
75000,comple

In [134]:
## evalueating the result and get test accuracy 
# Keep track of correct guesses in a confusion matrix
n_label = 2
confusion = torch.zeros(n_label, n_label)
#n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        

    return output

for whole_item in test_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    ### set model1 = False to use my own word2vec model
    review_tensor = review2tensor(review, model1= False)
    output = evaluate(review_tensor)
    
    label_pred = labelfromoutput(output)
    confusion[label][label_pred] += 1


In [135]:
confusion 

tensor([[17837.,  2130.],
        [ 5213., 14820.]])

In [136]:
print("the Test Accuracy for binary case with my own model is {}".format(accuracy_cf(confusion)))

the Test Accuracy for binary case with my own model is 0.8164250254631042


# Ternary  Case : My own word2vec model 

In [137]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 50
n_label = 3

rnn = RNN(300, n_hidden, n_label)

In [138]:
criterion = nn.NLLLoss()

learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()
    ### to prevent grandient explode
    torch.nn.utils.clip_grad_norm_(rnn.parameters(),0.5)

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [139]:
### train test split 
from sklearn.model_selection import train_test_split
import random
random.shuffle(ternary_data)
train_dataset, test_dataset = train_test_split(ternary_data, test_size=0.2,random_state=42)

In [140]:
import time
import math


#### train using train dataset 
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

correct = 0
index = 0
for whole_item in train_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    ## using my own word2vec model set model1 = False
    review_tensor = review2tensor(review, model1= False)
    
    output, loss = train(label_tensor, review_tensor)
    current_loss += loss
    index +=1 
    ### 
    y_pred = labelfromoutput(output)
    if y_pred  == label:
        correct +=1 
        
    if index % 5000 == 0:
        
        print("{},complete {}%, {}s,each average current loss is {:4f}".format(index,index/len(train_dataset)*100,\
                                                                             timeSince(start),current_loss/index))
        
        current_loss = 0 
        

5000,complete 2.5%, 0m 35ss,each average current loss is 1.062224
10000,complete 5.0%, 1m 11ss,each average current loss is 0.534274
15000,complete 7.5%, 1m 47ss,each average current loss is 0.356569
20000,complete 10.0%, 2m 22ss,each average current loss is 0.265135
25000,complete 12.5%, 2m 58ss,each average current loss is 0.214100
30000,complete 15.0%, 3m 34ss,each average current loss is 0.177453
35000,complete 17.5%, 4m 10ss,each average current loss is 0.150625
40000,complete 20.0%, 4m 46ss,each average current loss is 0.132481
45000,complete 22.5%, 5m 21ss,each average current loss is 0.118327
50000,complete 25.0%, 5m 56ss,each average current loss is 0.107954
55000,complete 27.500000000000004%, 6m 32ss,each average current loss is 0.099036
60000,complete 30.0%, 7m 7ss,each average current loss is 0.092669
65000,complete 32.5%, 7m 43ss,each average current loss is 0.086807
70000,complete 35.0%, 8m 18ss,each average current loss is 0.107239
75000,complete 37.5%, 8m 53ss,each aver

In [141]:
## evalueating the result and get test accuracy 
# Keep track of correct guesses in a confusion matrix
n_label = 3
confusion = torch.zeros(n_label, n_label)

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
    return output

for whole_item in test_dataset:
    
    label = whole_item[1]
    review = whole_item[0]
    #print(label)
    label_tensor = torch.tensor([label],dtype =torch.long)
    ### set model1 = False to use my own word2vec model
    review_tensor = review2tensor(review, model1= False)
    output = evaluate(review_tensor)
    
    label_pred = labelfromoutput(output)
    confusion[label][label_pred] += 1


In [143]:
confusion

tensor([[17102.,  2775.,   133.],
        [ 4448., 15434.,   216.],
        [ 5132.,  4621.,   139.]])

In [144]:
print("the Test Accuracy for Ternary case with my own model is {:4f}".format(accuracy_cf(confusion)))

the Test Accuracy for Ternary case with my own model is 0.653500


# 5 B) GRU  Binary Case: pretrained model (google_news300)

In [201]:
import torch
import torch.nn as nn

